# Sample Loading

Notebook to load Sample data into the databse

## Imports

In [1]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load

## Connection

In [2]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Panel id search

In [3]:
parent_table_name = 'panels'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_panel,name_panel,material_id_panel,description_panel,fabrication_id_panel
0,3,Juan_Ignacio_placeholder,15,This is a placeholder because we dont know fro...,2
1,4,Pegaso Piramide 7922843,16,The panel features multiple height levels rang...,3
2,5,Na_01,16,None,4
3,6,Na_02,16,None,5
4,7,Na_03,16,None,6
5,8,Na_04,16,None,7
6,9,Na_05,16,None,8
7,11,Na_07,16,None,10
8,12,Na_08,16,None,11
9,13,Na_09,16,None,12


In [28]:
#we set the id
panel_id = 14

## Sample data

Compulsory information:
- name: Name to identify the panel
- panel_id
- height: In mm
- width: In mm
- thickness: In mm
- keyhole: bool
- parallel_faces: bool

Suggested metadata:
- description

In [ ]:
panel_name = parent_data[parent_data['id_panel']==panel_id]['name_panel'].values[0]

names = [(panel_name+'_'+str(i)) for i in range(1,6)]

height = 150
width = 40
thickness = 5

keyhole = True

parallel_faces = True

for name in names:

    load.load_sample(conn,name,panel_id,height,width,thickness,keyhole,parallel_faces)

Sample 'Na_10_1' loaded with ID: 97
Sample 'Na_10_2' loaded with ID: 98
Sample 'Na_10_3' loaded with ID: 99
Sample 'Na_10_4' loaded with ID: 100
Sample 'Na_10_5' loaded with ID: 101


## Make sure insert was correct

In [30]:
data = qrs.get_data_metadata('samples')

data

,id_sample,name_sample,description_sample,panel_id_sample,height_sample,width_sample,thickness_sample,keyhole_sample,parallel_faces_sample,Missing_sample
0,9,JI_4,None,3,150.0 mm,50.0 mm,5.0 mm,True bool,True bool,NaN
1,10,JI_5,None,3,150.0 mm,40.0 mm,5.0 mm,True bool,True bool,NaN
2,11,JI_7,None,3,150.0 mm,40.0 mm,5.0 mm,True bool,True bool,NaN
3,12,JI_8,None,3,150.0 mm,40.0 mm,5.0 mm,True bool,True bool,NaN
4,13,JI_9,None,3,150.0 mm,40.0 mm,5.0 mm,True bool,True bool,NaN
...,...,...,...,...,...,...,...,...,...,...
78,97,Na_10_1,None,14,150 mm,40 mm,5 mm,True bool,True bool,NaN
79,98,Na_10_2,None,14,150 mm,40 mm,5 mm,True bool,True bool,NaN
80,99,Na_10_3,None,14,150 mm,40 mm,5 mm,True bool,True bool,NaN
81,100,Na_10_4,None,14,150 mm,40 mm,5 mm,True bool,True bool,NaN


In [32]:
conn.close()